### Calculating the Rates

The posterior on $\Lambda = R \langle VT \rangle$ for $n$ candidates can be expressed as:

$p(\Lambda|d) \propto p(R|\langle VT \rangle) p(\langle VT \rangle)e^{-\Lambda} \Lambda^n $

which when marginalised over $\langle VT \rangle$ gives $p(R|d)$. The upper limit $R_c$ on the rate with confidence $c$ is then the solution to:

$\int^{R_c}_0 dR p(R|d) = c$

For uniform prior on $\Lambda$, the rate at $90\%$ confidence is:
* $R^{n=0}_{90\%} = \frac{2.303}{\langle VT \rangle}$
* $R^{n=1}_{90\%} = \frac{3.88972}{\langle VT \rangle}$


* Please note here I am using the case of OR injections $\rightarrow$ meaning we are using union of injection time from both the pipeline runs

In [1]:
%pylab inline
%config InlineBackend.figure_format = 'retina'

Populating the interactive namespace from numpy and matplotlib


### Reading Modules

In [2]:
import pandas as pd
import numpy
from scipy.integrate import trapz, cumtrapz
import json
import numpy
import seaborn as sns
from scipy.integrate import cumtrapz
import pylab
pylab.style.use('seaborn-colorblind')

### Creating the Dataframe containing Injection bin wise Results

In [3]:
MassBin = numpy.arange(0,46)

M_tot = [120, 120, 120, 120, 120, 120, 150,
         200, 200, 200, 200, 220, 250, 300, 350,
         400, 400, 400, 400, 400, 440, 500, 600, 600, 800,
         120, 200, 400, 600, 800,
         120, 200, 400, 600, 800,
         200, 200, 200, 200, 
         400, 400, 400, 400, 600, 600, 800]

q = [1, 2, 4., 5, 7, 10, 2,
     1, 2, 4, 7, 10, 4, 2, 6,
     1, 2, 3, 4, 7, 10, 3/2, 1, 2, 1,
     1, 1, 1, 1, 1,
     1, 1, 1, 1, 1,
     1, 2, 4, 7,
     1, 2, 4, 7, 1, 2, 1]

chieff, chip = [], []
chieff += 25 * [0]
chieff += 5 * [0.8]
chieff += 5 * [-0.8]
chieff += [0.51, 0.14, 0.26, 0.32, 0.51, 0.14, 0.26, 0.32, 0.51, 0.14, 0.51]
BinKind = []
BinKind+= 46*['']
chip += 35 * [0]
chip += 11 * [0.42]


N_tot = [235438, 242893, 269575, 277772, 297997, 321276, 209975, 185246, 184217,
         193364, 201356, 192595, 171494, 157489, 141812, 170032, 144070, 124076,
         115944, 118781, 100269, 152056, 158088, 131182, 153438, 166859, 154551,
         139675, 138797, 148502, 253133, 205135, 187041, 180722, 175714, 155127,
         186574, 190711, 206275, 141121, 128394, 110405, 106686, 115607, 106521, 
         131695]
VT_tot = [340.078, 282.083, 162.518, 124.720, 79.686, 47.8479, 255.64, 255.64, 209.793,
          124.72, 63.1258, 34.1968, 97.2134, 124.72, 34.1968, 97.2134, 71.2662, 47.8479, 
          34.1968, 17.5403, 6.35876, 47.8479, 28.0926, 22.5271, 9.43448, 426.887, 399.686,
          200.413, 88.3472, 40.7967, 246.634, 246.634, 55.3055, 13.167, 3.94099, 307.595,
          264.552, 200.413, 124.72, 134.097, 106.251, 79.6861, 47.8478, 47.8478, 34.1968,
          17.5403]

In [4]:
df = pd.DataFrame()
df['MassBin'] = MassBin
df['Bin Kind'] = BinKind
df['M_tot'] = M_tot
df['q'] = q
df['chieff'] = chieff
df['chip'] = chip
df['N_tot'] = N_tot
df['VT_tot'] = VT_tot

In [5]:
sim_id=['SXS:BBH:0180, RIT:BBH:0198:n140, GT:0905', 
        'SXS:BBH:0169, RIT:BBH:0117:n140, GT:0446',
        'SXS:BBH:0182, RIT:BBH:0119:n140, GT:0454',
        'SXS:BBH:0056, RIT:BBH:0120:n140, GT:0906',
        'SXS:BBH:0298 RIT:BBH:Q10:n173, GT:0568',
        'SXS:BBH:0154, RIT:BBH:0068:n100',
        'SXS:BBH:0169, RIT:BBH:0117:n140, GT:0446',
        'SXS:BBH:0180, RIT:BBH:0198:n140, GT:0905',
        'SXS:BBH:0169, RIT:BBH:0117:n140, GT:0446',
        'SXS:BBH:0182, RIT:BBH:0119:n140, GT:0454',
        'SXS:BBH:0298 RIT:BBH:Q10:n173, GT:0568',
        'SXS:BBH:0154, RIT:BBH:0068:n100',
        'SXS:BBH:0182, RIT:BBH:0119:n140, GT:0454',
        'SXS:BBH:0169, RIT:BBH:0117:n140, GT:0446',
        'SXS:BBH:0181, RIT:BBH:0121:n140, GT:0604',
        'SXS:BBH:0180, RIT:BBH:0198:n140, GT:0905',
        'SXS:BBH:0169, RIT:BBH:0117:n140, GT:0446',
        'SXS:BBH:0030, RIT:BBH:0102:n140, GT:0453',
        'SXS:BBH:0182, RIT:BBH:0119:n140, GT:0454',
        'SXS:BBH:0298 RIT:BBH:Q10:n173, GT:0568',
        'RIT:BBH:Q10:n173, GT:0568',
        'RIT:BBH:0115:n140, GT:0477',
        'SXS:BBH:0180, RIT:BBH:0198:n140, GT:0905',
        'SXS:BBH:0169, RIT:BBH:0117:n140, GT:0446',
        'SXS:BBH:0180, RIT:BBH:0198:n140, GT:0905',
        'SXS:BBH:0230, RIT:BBH:0063:n100',
        'SXS:BBH:0230, RIT:BBH:0063:n100',
        'SXS:BBH:0230, RIT:BBH:0063:n100',
        'SXS:BBH:0230, RIT:BBH:0063:n100',
        'SXS:BBH:0230, RIT:BBH:0063:n100',
        'SXS:BBH:0154,RIT:BBH:0068:n100',
        'SXS:BBH:0154,RIT:BBH:0068:n100',
        'SXS:BBH:0154,RIT:BBH:0068:n100',
        'SXS:BBH:0154,RIT:BBH:0068:n100',
        'SXS:BBH:0154,RIT:BBH:0068:n100',
        'GT:0803',
        'GT:0872',
        'GT:0875',
        'GT:0888',
        'GT:0803',
        'GT:0872',
        'GT:0875',
        'GT:0888',
        'GT:0803',
        'GT:0872',
        'GT:0803',
        ]

In [6]:
z = [2.35, 2.00, 1.35, 1.15, 0.90, 0.70, 1.85, 1.85,
     1.60, 1.15, 0.80, 0.60, 1.00 , 1.15, 0.60, 1.0,
     0.85, 0.70, 0.60, 0.45, 0.30, 0.70, 0.55, 0.50,
     0.35, 2.95, 2.75, 1.55, 0.95, 0.65, 1.80, 1.45,
     0.75, 0.40, 0.25, 2.15, 1.90, 1.55, 1.15, 1.20, 
     1.05, 0.90, 0.70, 0.70, 0.60, 0.45
     ]

In [9]:
N_rec = [13201, 11758, 9327, 8543, 7768, 6908, 11523, 12669,
         11125, 8213, 6278, 5175, 7623, 10198, 5479, 10568,
         8682, 7160, 6469, 5423, 5655, 8138, 6646, 6187, 3412,
         16268, 16745, 14963, 10825, 5262, 11056, 10331, 8439,
         4421, 2990, 15088, 11752, 9433, 7635, 13375, 8202, 6226,
         5079, 7943, 5726, 1710]         

In [10]:
df['SIM_ID'] = sim_id
df['z'] = z
df['N_rec'] = N_rec
df['VT_sen'] = df['N_rec']/df['N_tot']*df['VT_tot']
df['R_n_0'] = 2.303/df['VT_sen']
df['R_n_1'] = 3.88972/df['VT_sen']

### Table containing all the details of the Injections Bin-Wise

In [11]:
df

,MassBin,Bin Kind,M_tot,q,chieff,chip,N_tot,VT_tot,SIM_ID,z,N_rec,VT_sen,R_n_0,R_n_1
0,0,,120,1.0,0.00,0.00,235438,340.07800,"SXS:BBH:0180, RIT:BBH:0198:n140, GT:0905",2.35,13201,19.068161,0.120777,0.203990
1,1,,120,2.0,0.00,0.00,242893,282.08300,"SXS:BBH:0169, RIT:BBH:0117:n140, GT:0446",2.00,11758,13.655115,0.168655,0.284854
2,2,,120,4.0,0.00,0.00,269575,162.51800,"SXS:BBH:0182, RIT:BBH:0119:n140, GT:0454",1.35,9327,5.622945,0.409572,0.691759
3,3,,120,5.0,0.00,0.00,277772,124.72000,"SXS:BBH:0056, RIT:BBH:0120:n140, GT:0906",1.15,8543,3.835818,0.600393,1.014052
4,4,,120,7.0,0.00,0.00,297997,79.68600,"SXS:BBH:0298 RIT:BBH:Q10:n173, GT:0568",0.90,7768,2.077205,1.108701,1.872574
5,5,,120,10.0,0.00,0.00,321276,47.84790,"SXS:BBH:0154, RIT:BBH:0068:n100",0.70,6908,1.028814,2.238500,3.780780
6,6,,150,2.0,0.00,0.00,209975,255.64000,"SXS:BBH:0169, RIT:BBH:0117:n140, GT:0446",1.85,11523,14.029002,0.164160,0.277263
7,7,,200,1.0,0.00,0.00,185246,255.64000,"SXS:BBH:0180, RIT:BBH:0198:n140, GT:0905",1.85,12669,17.483256,0.131726,0.222483
8,8,,200,2.0,0.00,0.00,184217,209.79300,"SXS:BBH:0169, RIT:BBH:0117:n140, GT:0446",1.60,11125,12.669553,0.181774,0.307013
9,9,,200,4.0,0.00,0.00,193364,124.72000,"SXS:BBH:0182, RIT:BBH:0119:n140, GT:0454",1.15,8213,5.297394,0.434742,0.734270


### Converting the table to Latex to copy and paste for paper

In [12]:
df = df.round(2)
headers = ['M_tot', 'q', 'chieff', 'chip', 'SIM_ID', 'z', 'VT_sen', 'R_n_0', 'R_n_1'] 
print(df.to_latex(index=False, columns=headers))

\begin{tabular}{rrrrlrrrr}
\toprule
 M\_tot &     q &  chieff &  chip &                                    SIM\_ID &     z &  VT\_sen &  R\_n\_0 &  R\_n\_1 \\
\midrule
   120 &   1.0 &    0.00 &  0.00 &  SXS:BBH:0180, RIT:BBH:0198:n140, GT:0905 &  2.35 &   19.07 &   0.12 &   0.20 \\
   120 &   2.0 &    0.00 &  0.00 &  SXS:BBH:0169, RIT:BBH:0117:n140, GT:0446 &  2.00 &   13.66 &   0.17 &   0.28 \\
   120 &   4.0 &    0.00 &  0.00 &  SXS:BBH:0182, RIT:BBH:0119:n140, GT:0454 &  1.35 &    5.62 &   0.41 &   0.69 \\
   120 &   5.0 &    0.00 &  0.00 &  SXS:BBH:0056, RIT:BBH:0120:n140, GT:0906 &  1.15 &    3.84 &   0.60 &   1.01 \\
   120 &   7.0 &    0.00 &  0.00 &    SXS:BBH:0298 RIT:BBH:Q10:n173, GT:0568 &  0.90 &    2.08 &   1.11 &   1.87 \\
   120 &  10.0 &    0.00 &  0.00 &           SXS:BBH:0154, RIT:BBH:0068:n100 &  0.70 &    1.03 &   2.24 &   3.78 \\
   150 &   2.0 &    0.00 &  0.00 &  SXS:BBH:0169, RIT:BBH:0117:n140, GT:0446 &  1.85 &   14.03 &   0.16 &   0.28 \\
   200 &   1.0 &    

### Converting table to Markdown to copy and paste in the wiki

In [13]:
print(df.to_markdown(index=False))

|   MassBin | Bin Kind   |   M_tot |    q |   chieff |   chip |   N_tot |   VT_tot | SIM_ID                                   |    z |   N_rec |   VT_sen |   R_n_0 |   R_n_1 |
|----------:|:-----------|--------:|-----:|---------:|-------:|--------:|---------:|:-----------------------------------------|-----:|--------:|---------:|--------:|--------:|
|         0 |            |     120 |  1   |     0    |   0    |  235438 |   340.08 | SXS:BBH:0180, RIT:BBH:0198:n140, GT:0905 | 2.35 |   13201 |    19.07 |    0.12 |    0.2  |
|         1 |            |     120 |  2   |     0    |   0    |  242893 |   282.08 | SXS:BBH:0169, RIT:BBH:0117:n140, GT:0446 | 2    |   11758 |    13.66 |    0.17 |    0.28 |
|         2 |            |     120 |  4   |     0    |   0    |  269575 |   162.52 | SXS:BBH:0182, RIT:BBH:0119:n140, GT:0454 | 1.35 |    9327 |     5.62 |    0.41 |    0.69 |
|         3 |            |     120 |  5   |     0    |   0    |  277772 |   124.72 | SXS:BBH:0056, RIT:BBH:0120:n140, GT